In [1]:
import os
import torch
import pandas as pd
import altair as alt

from src.baseline.baseline import train_df, figures_root
from src.finetune.finetuner import run_train, run_inference
from src.baseline.utils import calculate_class_distribution, calculate_metrics

2025-12-04 12:37:26.387266: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-12-04 12:37:26.414447: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-12-04 12:37:27.011735: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


✓ All random seeds set to 42
training files: ['train_en.csv', 'train_it.csv', 'train_es.csv']
Total training samples: 2988
CLASS DISTRIBUTION

Overall:
  Class 0 (NOT_RECLAMATORY): 2560 (85.7%)
  Class 1 (RECLAMATORY): 428 (14.3%)
  Total: 2988

Per Language:
  EN: Class 0=938, Class 1=88, Total=1026
  ES: Class 0=743, Class 1=133, Total=876
  IT: Class 0=879, Class 1=207, Total=1086




In [2]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /home/ubuntu/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
original_data = train_df
augmented_data = pd.read_csv("../data/augmented_multilingual_tweets.csv")
print(original_data.shape, augmented_data.shape)

(2988, 5) (5976, 8)


In [4]:
merged_data = pd.concat([original_data, augmented_data[list(original_data.columns)]], ignore_index=True)
print(merged_data.shape)

(8964, 5)


In [5]:
train_data = augmented_data

In [6]:
calculate_class_distribution(train_data)

CLASS DISTRIBUTION

Overall:
  Class 0 (NOT_RECLAMATORY): 5120 (85.7%)
  Class 1 (RECLAMATORY): 856 (14.3%)
  Total: 5976

Per Language:
  EN: Class 0=1622, Class 1=340, Total=1962
  ES: Class 0=1817, Class 1=295, Total=2112
  IT: Class 0=1681, Class 1=221, Total=1902




# Fine-Tuning on Merged Data

In [7]:
class Config:
    """Configuration for fine-tuning"""

    # Model configuration
    MODEL_NAME = "../fine_tuned_models_mlm/twitter-xlm-roberta-base/final_MLM_model/model" #"cardiffnlp/twitter-xlm-roberta-base"  # Base model
    NUM_LABELS = 2  # Binary classification
    MAX_LENGTH = 128  # Maximum sequence length
    NUM_FROZEN_LAYERS = 3  # Number of initial layers to freeze (0 = only train classification head)

    # Training configuration
    LEARNING_RATE = 2e-5
    WEIGHT_DECAY = 0.01
    NUM_EPOCHS = 10
    BATCH_SIZE = 8
    GRADIENT_ACCUMULATION_STEPS = 2
    WARMUP_RATIO = 0.1  # Warmup as % of total steps

    # Early stopping
    PATIENCE = 3
    EVAL_STRATEGY = "epoch"  # Evaluate at end of each epoch

    # Cross-validation
    N_SPLITS = 5
    TRAIN_RATIO = 0.8  # 80% for training from each fold
    VAL_RATIO = 0.2  # 20% for validation from each fold

    # Dynamic undersampling
    DYNAMIC_UNDERSAMPLE = False  # Balance classes per epoch

    # Model saving
    MAX_MODELS_TO_SAVE = 2
    OUTPUT_DIR = "../fine_tuned_models"
    RESULTS_DIR = "../results/roberta-fine-tune/augumented/"

    # Device
    DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Training would be start on the device: {DEVICE}")

Training would be start on the device: cuda


In [8]:
best_checkpoint_path = run_train(train_data, Config)

2025-12-04 12:37:27,861 - INFO - Fold 0: Train=3824, Val=956
2025-12-04 12:37:27,861 - INFO -   Train label dist: {0: 3276, 1: 548}
2025-12-04 12:37:27,862 - INFO -   Train lang dist: {'es': 1352, 'en': 1255, 'it': 1217}
2025-12-04 12:37:27,866 - INFO - Fold 1: Train=3824, Val=957
2025-12-04 12:37:27,866 - INFO -   Train label dist: {0: 3277, 1: 547}
2025-12-04 12:37:27,867 - INFO -   Train lang dist: {'es': 1352, 'en': 1255, 'it': 1217}
2025-12-04 12:37:27,870 - INFO - Fold 2: Train=3824, Val=957
2025-12-04 12:37:27,871 - INFO -   Train label dist: {0: 3277, 1: 547}
2025-12-04 12:37:27,871 - INFO -   Train lang dist: {'es': 1352, 'en': 1256, 'it': 1216}
2025-12-04 12:37:27,875 - INFO - Fold 3: Train=3824, Val=957
2025-12-04 12:37:27,875 - INFO -   Train label dist: {0: 3278, 1: 546}
2025-12-04 12:37:27,876 - INFO -   Train lang dist: {'es': 1352, 'en': 1255, 'it': 1217}
2025-12-04 12:37:27,880 - INFO - Fold 4: Train=3824, Val=957
2025-12-04 12:37:27,880 - INFO -   Train label dist: {0


FOLD 0:
  Train: 548 positive samples
  Val:   137 positive samples


Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at ../fine_tuned_models_mlm/twitter-xlm-roberta-base/final_MLM_model/model and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
2025-12-04 12:37:28,690 - INFO - Froze: Embeddings + First 3 Encoder Layers
2025-12-04 12:37:28,691 - INFO - Trainable: Classification Head + Remaining Encoder Layers
2025-12-04 12:37:28,691 - INFO - Trainable parameters: 64,382,978 / 278,045,186 (23.16%)
2025-12-04 12:37:28,692 - INFO - Label weights: {0: 0.5836385836385837, 1: 3.489051094890511}
2025-12-04 12:37:28,692 - INFO - Language weights: {'es': 0.9409641154628656, 'en': 1.0136920192078043, 'it': 1.0453438653293299}
2025-12-04 12:37:28,692 - INFO - Pos weight (for BCE): 5.9781
2025-12-04 12:37:28,693 - INFO - 
E


FOLD 1:
  Train: 547 positive samples
  Val:   138 positive samples


Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at ../fine_tuned_models_mlm/twitter-xlm-roberta-base/final_MLM_model/model and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
2025-12-04 12:38:27,829 - INFO - Froze: Embeddings + First 3 Encoder Layers
2025-12-04 12:38:27,829 - INFO - Trainable: Classification Head + Remaining Encoder Layers
2025-12-04 12:38:27,830 - INFO - Trainable parameters: 64,382,978 / 278,045,186 (23.16%)
2025-12-04 12:38:27,832 - INFO - Label weights: {0: 0.5834604821483064, 1: 3.495429616087751}
2025-12-04 12:38:27,832 - INFO - Language weights: {'es': 0.9409641154628656, 'en': 1.0136920192078043, 'it': 1.0453438653293299}
2025-12-04 12:38:27,832 - INFO - Pos weight (for BCE): 5.9909
2025-12-04 12:38:27,834 - INFO - 
E

...

Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at ../fine_tuned_models_mlm/twitter-xlm-roberta-base/final_MLM_model/model and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
2025-12-04 12:40:56,503 - INFO - Froze: Embeddings + First 3 Encoder Layers
2025-12-04 12:40:56,504 - INFO - Trainable: Classification Head + Remaining Encoder Layers
2025-12-04 12:40:56,504 - INFO - Trainable parameters: 64,382,978 / 278,045,186 (23.16%)
2025-12-04 12:40:56,505 - INFO - Label weights: {0: 0.5834604821483064, 1: 3.495429616087751}
2025-12-04 12:40:56,506 - INFO - Language weights: {'es': 0.9409476243674836, 'en': 1.0128671880134061, 'it': 1.0461851876191102}
2025-12-04 12:40:56,506 - INFO - Pos weight (for BCE): 5.9909
2025-12-04 12:40:56,507 - INFO - 
E


FOLD 3:
  Train: 546 positive samples
  Val:   138 positive samples


Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at ../fine_tuned_models_mlm/twitter-xlm-roberta-base/final_MLM_model/model and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
2025-12-04 12:41:54,647 - INFO - Froze: Embeddings + First 3 Encoder Layers
2025-12-04 12:41:54,647 - INFO - Trainable: Classification Head + Remaining Encoder Layers
2025-12-04 12:41:54,648 - INFO - Trainable parameters: 64,382,978 / 278,045,186 (23.16%)
2025-12-04 12:41:54,649 - INFO - Label weights: {0: 0.5832824893227577, 1: 3.501831501831502}
2025-12-04 12:41:54,649 - INFO - Language weights: {'es': 0.9409641154628656, 'en': 1.0136920192078043, 'it': 1.0453438653293299}
2025-12-04 12:41:54,649 - INFO - Pos weight (for BCE): 6.0037
2025-12-04 12:41:54,651 - INFO - 
E


FOLD 4:
  Train: 547 positive samples
  Val:   138 positive samples


Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at ../fine_tuned_models_mlm/twitter-xlm-roberta-base/final_MLM_model/model and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
2025-12-04 12:42:52,745 - INFO - Froze: Embeddings + First 3 Encoder Layers
2025-12-04 12:42:52,746 - INFO - Trainable: Classification Head + Remaining Encoder Layers
2025-12-04 12:42:52,746 - INFO - Trainable parameters: 64,382,978 / 278,045,186 (23.16%)
2025-12-04 12:42:52,747 - INFO - Label weights: {0: 0.5834604821483064, 1: 3.495429616087751}
2025-12-04 12:42:52,748 - INFO - Language weights: {'es': 0.9416953224870754, 'en': 1.0129222776114961, 'it': 1.045382399901429}
2025-12-04 12:42:52,748 - INFO - Pos weight (for BCE): 5.9909
2025-12-04 12:42:52,749 - INFO - 
Ep

### Visualizing Fine-Tune Metrics

In [9]:
df = pd.read_csv(os.path.join(Config.RESULTS_DIR, "training_results.csv"))

In [10]:
f1_plot = alt.Chart(df).mark_line(point=True, size=3).encode(
    x=alt.X('epoch:Q', title='Epoch'),
    y=alt.Y('overall_macro_f1:Q', title='Macro F1 Score', scale=alt.Scale(domain=[0.3, 0.8])),
    color=alt.Color('fold:N', title='Fold'),
    tooltip=['fold:N', 'epoch:Q', alt.Tooltip('overall_macro_f1:Q', format='.4f')]
).properties(width=600, height=300, title='Overall F1 Score by Fold')
f1_plot.save(os.path.join(figures_root, 'f1_vs_fold_augmented.svg'))
os.path.join(figures_root, 'f1_vs_fold_augmented.svg')

'../figures/f1_vs_fold_augmented.svg'

![](../figures/f1_vs_fold_augmented.svg)

In [11]:
loss_data = df[['fold', 'epoch', 'train_loss', 'val_loss']].melt(
    id_vars=['fold', 'epoch'], var_name='loss_type', value_name='loss'
)

loss_plot = alt.Chart(loss_data).mark_line(point=True).encode(
    x='epoch:Q', y='loss:Q', color='loss_type:N', strokeDash='fold:N',
    tooltip=['fold:N', 'epoch:Q', 'loss_type:N', alt.Tooltip('loss:Q', format='.4f')]
).properties(width=600, height=300, title='Training & Validation Loss')

loss_plot.save(os.path.join(figures_root, 'loss_augmented.svg'))
os.path.join(figures_root, 'loss_augmented.svg')

'../figures/loss_augmented.svg'

![](../figures/loss_augmented.svg)

In [12]:
lang_data = df[['fold', 'epoch', 'en_macro_f1', 'es_macro_f1', 'it_macro_f1']].melt(
    id_vars=['fold', 'epoch'], var_name='language', value_name='f1'
)
lang_data['language'] = lang_data['language'].str.replace('_macro_f1', '').str.upper()

lang_plot = alt.Chart(lang_data).mark_line(point=True).encode(
    x='epoch:Q', y=alt.Y('f1:Q', scale=alt.Scale(domain=[0.4, 0.95])), color='language:N', strokeDash='fold:N',
    tooltip=['fold:N', 'epoch:Q', 'language:N', alt.Tooltip('f1:Q', format='.4f')]
).properties(width=600, height=300, title='F1 by Language')

lang_plot.save(os.path.join(figures_root, 'f1_vs_lang_augmented.svg'))
os.path.join(figures_root, 'f1_vs_lang_augmented.svg')

'../figures/f1_vs_lang_augmented.svg'

![](../figures/f1_vs_lang_augmented.svg)

In [13]:
metrics_data = df[['fold', 'epoch', 'overall_macro_precision', 'overall_macro_recall', 'overall_macro_f1']].melt(
    id_vars=['fold', 'epoch'], var_name='metric', value_name='value'
)
metrics_data['metric'] = metrics_data['metric'].str.replace('overall_macro_', '').str.capitalize()
metrics_data['fold_epoch'] = 'F' + metrics_data['fold'].astype(str) + ':E' + metrics_data['epoch'].astype(str)

heatmap = alt.Chart(metrics_data).mark_rect().encode(
    x='fold_epoch:O', y='metric:N',
    color=alt.Color('value:Q', scale=alt.Scale(scheme='viridis')),
    tooltip=['fold_epoch:N', 'metric:N', alt.Tooltip('value:Q', format='.4f')]
).properties(width=700, height=150, title='Precision/Recall/F1 Heatmap')

heatmap.save(os.path.join(figures_root, 'fold_vs_epoch_augmented.svg'))
os.path.join(figures_root, 'fold_vs_epoch_augmented.svg')

'../figures/fold_vs_epoch_augmented.svg'

![](../figures/fold_vs_epoch_augmented.svg)

# Inference

In [14]:
class InferenceConfig:
    MODEL_NAME = "cardiffnlp/twitter-xlm-roberta-base"
    CHECKPOINT_PATH = best_checkpoint_path
    MAX_LENGTH = 128
    BATCH_SIZE = 32
    NUM_LABELS = 2
    DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [15]:
results = run_inference(train_df, InferenceConfig)

Running inference on 2988 samples...
Device: cuda


Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at cardiffnlp/twitter-xlm-roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Loaded checkpoint from: ../fine_tuned_models/checkpoints/fold_1_epoch_7_f1_0.7040.pt


Inference: 100%|██████████| 94/94 [00:02<00:00, 39.37it/s]



INFERENCE RESULTS ON TRAIN DATA

Overall Metrics:
  Macro Precision: 0.6085
  Macro Recall:    0.6135
  Macro F1:        0.6109

Per-Language Metrics:
  EN:
    Precision: 0.5305
    Recall:    0.5780
    F1:        0.5102
  ES:
    Precision: 0.7147
    Recall:    0.6831
    F1:        0.6966
  IT:
    Precision: 0.8901
    Recall:    0.6257
    F1:        0.6594
